## Baseline, randomly guessing that 20% of the data is popular

In [3]:
import random
my_list = [True for x in range(2)] +  [False for x in range(8)]
accuracy_baseline = np.zeros(shape=(1,9))
recall_baseline = np.zeros(shape=(1,9))
precision_baseline = np.zeros(shape=(1,9))
f1_baseline = np.zeros(shape=(1,9))

In [163]:
df_sf_temp = df_sf_2017.copy()
start_month = 1
end_month = 4

while end_month <13:
    y_test = df_sf_temp[df_sf_temp['month'] == end_month]['popular']
    y_pred = pd.Series(random.choice(my_list) for x in range(y_test.size))
    accuracy_baseline[0][start_month-1] = accuracy_score(y_test, y_pred)
    recall_baseline[0][start_month-1] = recall_score(y_test, y_pred)
    precision_baseline[0][start_month-1] = precision_score(y_test, y_pred)
    f1_baseline[0][start_month-1] = f1_score(y_test, y_pred)
    start_month += 1
    end_month += 1

print(accuracy_baseline.mean())
print(recall_baseline.mean()) 
print(precision_baseline.mean())
print(f1_baseline.mean())


0.6760991542390486
0.19911653217717348
0.2076371303722557
0.20267624173813048


In [2]:
import pandas as pd
import numpy as np
import pickle
from datetime import datetime as dt
from sklearn.ensemble import RandomForestClassifier

pd.set_option("display.max_colwidth",999)
pd.set_option("display.max_rows",999)
pd.set_option("display.max_columns",999)

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [62]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [86]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

# Load pickle data

In [9]:
# df_sf_2017 = pickle.load(open('../data_sf_2017.p', 'rb'))

In [10]:
df_sf_2017 = pd.read_json('df_sf_2017')

In [11]:
len(df_sf_2017)

117262

# NLP

In [56]:
import nltk
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import time
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score

In [13]:
df_sf_2017.description.fillna(value='None', inplace=True)

## Test

In [135]:
test1 = df_sf_2017['description'].iloc[0:2]

In [136]:
tf_vectorizer_train = CountVectorizer(analyzer=process_text).fit(test1)

In [137]:
tf_vectorizer_train.vocabulary_

{'this': 95,
 'room': 81,
 'perfect': 68,
 'locat': 52,
 'enjoy': 27,
 'peac': 66,
 'environ': 29,
 'want': 107,
 'close': 19,
 'shop': 89,
 'restaur': 79,
 'nightlif': 62,
 'it': 46,
 'one': 64,
 'block': 13,
 'j': 47,
 'train': 98,
 '20m': 3,
 'ride': 80,
 'downtown': 24,
 '3': 4,
 'mission': 57,
 'the': 93,
 'apart': 8,
 '2': 2,
 'bedroom': 11,
 '1': 0,
 'recent': 75,
 'remodel': 76,
 'bathroom': 10,
 'there': 94,
 'modern': 58,
 'kitchen': 48,
 'dishwash': 22,
 'washer': 108,
 'dryer': 25,
 'build': 15,
 'use': 103,
 'best': 12,
 'hill': 43,
 'climb': 18,
 'get': 37,
 'feel': 31,
 'light': 49,
 'airi': 6,
 'minimalist': 55,
 'terrarium': 92,
 'various': 105,
 'plant': 70,
 'scatter': 85,
 'throughout': 96,
 'you': 111,
 'access': 5,
 'entir': 28,
 'place': 69,
 'includ': 45,
 'live': 51,
 'dine': 21,
 'garag': 35,
 'share': 88,
 'u': 102,
 'two': 101,
 'i': 44,
 'happi': 41,
 'help': 42,
 'anyth': 7,
 'need': 60,
 'servic': 87,
 'dog': 23,
 'request': 77,
 'guest': 40,
 'plea': 71,

In [125]:
text = 'walk you i san are to .be hippo geese. geese goose francisco walked'

stop_words = stopwords.words("english")
stop_words.append('san')
stop_words.append('francisco')
tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(text)
stemmed = []
snowball = SnowballStemmer("english")
for item in tokens:
    if item not in stop_words:
        print(item)
        stemmed.append(snowball.stem(item))
print(stemmed)

walk
hippo
geese
geese
goose
walked
['walk', 'hippo', 'gees', 'gees', 'goos', 'walk']


In [107]:
%%time

def process_text(text):
    stop_words = stopwords.words("english")
    stop_words.append('san')
    stop_words.append('francisco')
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    
    stemmed = []
    snowball = SnowballStemmer("english")
    for item in tokens:
        if item not in stop_words:
            stemmed.append(snowball.stem(item))
        
    lemmatized = []
    wordnet = WordNetLemmatizer()
    for item in stemmed:
        lemmatized.append(wordnet.lemmatize(item))
    
    return lemmatized
    

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.68 µs


## split data - running NLP on description column

In [15]:
def split_data(start_month, end_month):
    df_X_train = df_sf_2017[(df_sf_2017['month'] >= start_month) & (df_sf_2017['month'] < end_month)]['description']
    y_train = df_sf_2017[(df_sf_2017['month'] >= start_month) & (df_sf_2017['month'] < end_month)]['popular']

    df_X_test = df_sf_2017[df_sf_2017['month'] == end_month]['description']
    y_test = df_sf_2017[df_sf_2017['month'] == end_month]['popular']
    
    return df_X_train, y_train, df_X_test, y_test

## Run CountVectorizer

In [71]:
%%time
def run_tf_vec(df_X_train, df_X_test):
    tf_vectorizer_train = CountVectorizer(analyzer=process_text).fit(df_X_train)
    X_train = tf_vectorizer_train.transform(df_X_train)
    tf_vectorizer_test = CountVectorizer(analyzer=process_text, vocabulary=tf_vectorizer_train.vocabulary_).fit(df_X_test)
    X_test = tf_vectorizer_test.transform(df_X_test)
    return X_train, X_test, tf_vectorizer_train

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 18.6 µs


In [95]:
X_train

<26526x12784 sparse matrix of type '<class 'numpy.int64'>'
	with 1848230 stored elements in Compressed Sparse Row format>

In [72]:
def predict_tf_nb(X_train, y_train, X_test, y_test):
    from sklearn.naive_bayes import MultinomialNB
    nb = MultinomialNB()
    nb.fit(X_train, y_train)
    pickle.dump(nb, open('nb_model'+ str(model_num) + '.p', 'wb'))
    preds = nb.predict(X_test)
    scores_tf_nb[0][model_num] = accuracy_score(y_test, preds)
    scores_tf_nb[1][model_num] = recall_score(y_test, nb.predict(X_test))
    scores_tf_nb[2][model_num] = precision_score(y_test, nb.predict(X_test))
    scores_tf_nb[3][model_num] = f1_score(y_test, nb.predict(X_test))
    return scores_tf_nb

## With the CountVectorizer, run with RandomForest 

In [73]:
def predict_tf_rf(X_train, y_train, X_test, y_test):
    rf = RandomForestClassifier(n_estimators = 10, n_jobs = -1, random_state=0, class_weight = {0:.95, 1:.05})
    rf.fit(X_train, y_train)
    predicted = rf.predict(X_test)
    pickle.dump(rf, open('rf_nlp_countvec_50' + str(model_num) + '.p', 'wb'))
    scores_tf_rf[0][model_num] = accuracy_score(y_test, predicted)
    scores_tf_rf[1][model_num] = recall_score(y_test, predicted)
    scores_tf_rf[2][model_num] = precision_score(y_test, predicted)
    scores_tf_rf[3][model_num] = f1_score(y_test, predicted)
    return scores_tf_rf

In [19]:
scores_tf_nb = np.zeros(shape=(4,9))
scores_tf_rf = np.zeros(shape=(4,9))

model_num = 0
start_month = 1
end_month = 4

In [22]:
%%time
while end_month <13:
    df_X_train, y_train, df_X_test, y_test = split_data(start_month, end_month)
    X_train, X_test, tf_vectorizer_train = run_tf_vec(df_X_train, df_X_test)
    scores_tf_nb = predict_tf_nb(X_train, y_train, X_test, y_test)
    scores_tf_rf = predict_tf_rf(X_train, y_train, X_test, y_test)
    print(start_month, end_month, model_num)
    model_num += 1
    start_month += 1
    end_month += 1
    print('tf_nb')
    print(scores_tf_nb)
    print('tf_rf')
    print(scores_tf_rf)
    break

1 4 0
tf_nb
[[0.8589493  0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.66435768 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.60320183 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.63230447 0.         0.         0.         0.         0.
  0.         0.         0.        ]]
tf_rf
[[0.95539717 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.80730479 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.9398827  0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.86856369 0.         0.         0.         0.         0.
  0.         0.         0.        ]]
CPU times: user 2min 40s, sys: 1.21 s, total: 2min 42s
Wall time: 2min 35s


In [14]:
accuracy_2017_tf_nb = np.mean(scores_tf_nb[0])
recall_2017_tf_nb = np.mean(scores_tf_nb[1])
precision_2017_tf_nb = np.mean(scores_tf_nb[2])
f1_score_2017_tf_nb = np.mean(scores_tf_nb[3])
print(accuracy_2017_tf_nb)
print(recall_2017_tf_nb)
print(precision_2017_tf_nb)
print(f1_score_2017_tf_nb)

0.8455697882468312
0.6745968354038901
0.6159944961116426
0.6435671085940798


In [15]:
accuracy_2017_tf_rf = np.mean(scores_tf_rf[0])
recall_2017_tf_rf = np.mean(scores_tf_rf[1])
precision_2017_tf_rf = np.mean(scores_tf_rf[2])
f1_score_2017_tf_rf = np.mean(scores_tf_rf[3])
print(accuracy_2017_tf_rf)
print(recall_2017_tf_rf)
print(precision_2017_tf_rf)
print(f1_score_2017_tf_rf)

0.946939377339736
0.7846092062627434
0.9502124982965571
0.8590649087408558


## Try running with TF-IDF

In [74]:
%%time
def run_tf_idf_vec(df_X_train, df_X_test):
    tf_idf_vectorizer_train = TfidfVectorizer(analyzer=process_text)
    X_train2 = tf_idf_vectorizer_train.fit_transform(df_X_train).toarray()
    tf_idf_vectorizer_test = TfidfVectorizer(analyzer=process_text, vocabulary=tf_idf_vectorizer_train.vocabulary_)
    X_test2 = tf_idf_vectorizer_test.fit_transform(df_X_test).toarray()
    return X_train2, X_test2, tf_idf_vectorizer_train

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.78 µs


In [76]:
def predict_tf_idf_nb(X_train2, y_train, X_test2, y_test):
    from sklearn.naive_bayes import GaussianNB
    nb = GaussianNB()
    nb.fit(X_train2, y_train)
    # pickle.dump(nb, open('nb_model_guassian.p', 'wb'))
    preds = nb.predict(X_test2)
    scores_tf_idf_nb[0][model_num] = accuracy_score(y_test, preds)
    scores_tf_idf_nb[1][model_num] = recall_score(y_test, preds)
    scores_tf_idf_nb[2][model_num] = precision_score(y_test, preds)
    scores_tf_idf_nb[3][model_num] = f1_score(y_test, preds)
    return scores_tf_idf_nb

## With the TF-IDF, run with RandomForest 

In [77]:
def predict_tf_idf_rf(X_train2, y_train, X_test2, y_test):
    rf = RandomForestClassifier(n_estimators = 10, n_jobs=-1, random_state=0, class_weight = {0:.95, 1:.05})
    rf.fit(X_train2, y_train)
    predicted = rf.predict(X_test2)
#     pickle.dump(rf, open('rf_nlp_50.p', 'wb'))
    scores_tf_idf_rf[0][model_num] = accuracy_score(y_test, predicted)
    scores_tf_idf_rf[1][model_num] = recall_score(y_test, predicted)
    scores_tf_idf_rf[2][model_num] = precision_score(y_test, predicted)
    scores_tf_idf_rf[3][model_num] = f1_score(y_test, predicted)
    return scores_tf_idf_rf

In [78]:
scores_tf_idf_nb = np.zeros(shape=(4,9))
scores_tf_idf_rf = np.zeros(shape=(4,9))

model_num = 0
start_month = 1
end_month = 4

In [79]:
%%time
while end_month <13:
    df_X_train, y_train, df_X_test, y_test = split_data(start_month, end_month) 
    X_train2, X_test2, tf_idf_vectorizer_train = run_tf_idf_vec(df_X_train, df_X_test)
    scores_tf_idf_nb = predict_tf_idf_nb(X_train2, y_train, X_test2, y_test)
    scores_tf_idf_rf = predict_tf_idf_rf(X_train2, y_train, X_test2, y_test)
    print(start_month, end_month, model_num)
    model_num += 1
    start_month += 1
    end_month += 1
    print('tf_idf_nb')
    print(scores_tf_idf_nb)
    print('tf_idf_rf')
    print(scores_tf_idf_rf)
    break

1 4 0
tf_idf_nb
[[0.61328888 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.93702771 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.31313131 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.46940063 0.         0.         0.         0.         0.
  0.         0.         0.        ]]
tf_idf_rf
[[0.94643062 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.76448363 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.9295559  0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.83897719 0.         0.         0.         0.         0.
  0.         0.         0.        ]]
CPU times: user 6min 52s, sys: 3.94 s, total: 6min 56s
Wall time: 2min 24s


In [30]:
accuracy_2017_tf_idf_nb = np.mean(scores_tf_idf_nb[0])
recall_2017_tf_idf_nb = np.mean(scores_tf_idf_nb[1])
precision_2017_tf_idf_nb = np.mean(scores_tf_idf_nb[2])
f1_score_2017_tf_idf_nb = np.mean(scores_tf_idf_nb[3])
print(accuracy_2017_tf_idf_nb)
print(recall_2017_tf_idf_nb)
print(precision_2017_tf_idf_nb)
print(f1_score_2017_tf_idf_nb)

0.6223984688066541
0.9360869500755025
0.3467272958014258
0.5052056786500755


In [31]:
accuracy_2017_tf_idf_rf = np.mean(scores_tf_idf_rf[0])
recall_2017_tf_idf_rf = np.mean(scores_tf_idf_rf[1])
precision_2017_tf_idf_rf = np.mean(scores_tf_idf_rf[2])
f1_score_2017_tf_idf_rf = np.mean(scores_tf_idf_rf[3])
print(accuracy_2017_tf_idf_rf)
print(recall_2017_tf_idf_rf)
print(precision_2017_tf_idf_rf)
print(f1_score_2017_tf_idf_rf)

0.9383264864764923
0.7493591861607887
0.9398408080684738
0.8334356589996533


In [ ]:
# Do I need k means? Or just top twenty words for popular and top twenty words for not popular

## Kmeans

In [23]:
from sklearn.cluster import KMeans
from collections import Counter
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram

In [80]:
features =  tf_idf_vectorizer_train.get_feature_names()

In [27]:
features[150:200]

['1300',
 '1300sqft',
 '1304',
 '1315',
 '132',
 '135',
 '1350',
 '135cm',
 '136',
 '137',
 '1370123',
 '13724120',
 '139',
 '1394',
 '1396',
 '13ft',
 '13in',
 '13lb',
 '13mn',
 '13th',
 '13x12',
 '14',
 '140',
 '1400',
 '1400ft²',
 '1400sf',
 '1408',
 '140sq',
 '140sqm',
 '14154147',
 '143',
 '144',
 '145',
 '1450',
 '14611926',
 '1474',
 '149',
 '14ft',
 '14l',
 '14min',
 '14r',
 '14sqm',
 '14th',
 '14x',
 '14x12',
 '15',
 '150',
 '1500',
 '1500ft',
 '1500sf']

In [85]:
%%time
kmeans = KMeans(n_clusters=2, n_jobs=-1)
kmeans.fit(X_train)

CPU times: user 736 ms, sys: 264 ms, total: 1e+03 ms
Wall time: 3min 50s


In [96]:
%%time
# TF-IDF vectorizer
top_centroids = kmeans.cluster_centers_.argsort()[:,-1:-60:-1]
print("top features (words) for each cluster:")
for num, centroid in enumerate(top_centroids):
    print("%d: %s" % (num, ", ".join(features[i] for i in centroid)))


top features (words) for each cluster:
0: place, walk, locat, park, san, room, francisco, the, restaur, street, block, close, apart, kitchen, my, bedroom, citi, neighborhood, you, bed, we, i, great, access, sf, one, this, 2, view, travel, shop, privat, love, home, away, mission, downtown, 1, live, bart, bathroom, busi, space, minut, bar, hous, full, squar, good, share, coupl, it, check, build, stay, avail, distanc, area, heart
1: room, the, bedroom, park, kitchen, bed, live, locat, san, apart, walk, francisco, privat, bathroom, home, one, 2, access, block, street, neighborhood, i, floor, this, hous, restaur, full, we, larg, 1, guest, space, two, citi, area, queen, view, great, there, you, away, size, comfort, it, quiet, shop, mission, avail, share, also, beauti, sf, includ, stay, tv, dine, minut, unit, bath
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.13 ms


In [89]:
%%time
# TF vectorizer
top_centroids = kmeans.cluster_centers_.argsort()[:,-1:-50:-1]
print("top features (words) for each cluster:")
for num, centroid in enumerate(top_centroids):
    print("%d: %s" % (num, ", ".join(features[i] for i in centroid)))


top features (words) for each cluster:
0: place, walk, locat, park, san, room, francisco, the, restaur, street, block, close, apart, kitchen, my, bedroom, citi, neighborhood, you, bed, we, i, great, access, sf, one, this, 2, view, travel, shop, privat, love, home, away, mission, downtown, 1, live, bart, bathroom, busi, space, minut, bar, hous, full, squar, good
1: room, the, bedroom, park, kitchen, bed, live, locat, san, apart, walk, francisco, privat, bathroom, home, one, 2, access, block, street, neighborhood, i, floor, this, hous, restaur, full, we, larg, 1, guest, space, two, citi, area, queen, view, great, there, you, away, size, comfort, it, quiet, shop, mission, avail, share
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 1.83 ms
